In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Enter NCRB Individual PDF url
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-1A.8_2019.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
# Download report
fileName = wget.download(pdf_url,fileName)

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-1A.8_2019.pdf'

In [5]:
tables = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "31.26300484652666,675.6463402880256,572.8344426494347,212.94536985142634"
                              ],
                          columns=[
                              "47.134991922455576,167.37731825525043,216.43618739903073,258.7614862681745,298.2009693053312,328.9830048465267,368.4224878836834,411.70972536348955,450.1872697899839,488.66481421647825,527.1423586429726"
                              ],
                              row_tol=13,
                          split_text=True,
                          strip_text='\n',
                          pages="all"
                         )

In [6]:
tables

<TableList n=1>

In [7]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [8]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)
# tables[0].df

In [9]:
# Splits the second column into 1st column index and 2nd column test when faulty/required
def split_by_first_space(df, strIndex, stpIndex):
    df = df.iloc[strIndex:stpIndex+1]
    for index, each in df.iterrows():
        space = each[1].find(' ')
        each[0] = each[1][:space]
        each[1] = each[1][space:]

In [10]:
split_by_first_space(tables[0].df,7,8)

In [11]:
def mergeRows(df, strIndex, stpIndex):
    DF = df.loc[strIndex:stpIndex]
    df = DF.T
    l = []
    for index, each in df.iterrows():
        output = ''
        for ind, cell in each.iteritems():
            output += (str(cell) + ' ')
#         if(output == ''):
#             continue
        l.append(output.strip())
    l = pd.Series(l,name=str(strIndex))
    DF.loc[strIndex] = l

In [12]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [13]:
def set_Column(df):
    column_list = [
        ("Sl. No.",''),
        ("Cause",''), 
        ("No. of Cases",''),
        ('Persons Injured','Male'),
        ('Persons Injured','Female'),
        ('Persons Injured','Transgender'),
        ('Persons Injured','Total'),
        ('Persons Died','Male'),
        ('Persons Died','Female'),
        ('Persons Died','Transgender'),
        ('Persons Died','Total'),
        ('Percentage Share (Cases)','')
    ]
    df.columns = pd.MultiIndex.from_tuples(column_list)

In [14]:
for i in range(len(tables)):
    set_Column(tables[i].df)

In [15]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

# OR Print a simgle one according to your choice
# tables[23].df

In [16]:
# Reset Index before saving file, for better formatting in RAW CSV
tables[0].df.to_csv("Cause – wise Distribution of Road Accidents and Unmanned Railway Crossing Accidents during 2019 (All India).csv",index=False)